In [1]:
import pandas as pd
import math as mt

In [ ]:
event_training = pd.read_csv('training.csv')
event_test = pd.read_csv('test.csv')

# vamos a agregar otros features relacionados con el tiempo para cada usuario.

# primero calculamos que tan lejos esta el ultimo evento de cada tipo para cada usuario con respecto a la fecha limite (31 de mayo del 2018 a las 23:59:59)

In [ ]:
#primero agrego la columna date al set de datos
event_training['date'] = pd.to_datetime(event_training['timestamp'])

In [ ]:
# primero para viewed product, a los que no realizaron este evento les pongo -1, como manera de diferenciar

In [ ]:
def calcularDistancia(dataframe, evento, nombre):
    fecha_limite=pd.to_datetime('2018-05-31 23:59:59')
    usuarios_con_evento = dataframe.groupby('person').filter(lambda x: (x['event']==evento).any())\
                    .reset_index(drop=True)
    usuarios_sin_evento = dataframe.groupby('person').filter(lambda x: (x['event']!=evento).all())\
                    .reset_index(drop=True)
    tabla = usuarios_sin_evento.loc[:,['person','timestamp']].drop_duplicates(subset='person',keep='first')
    tabla.columns=['person',nombre]
    tabla[nombre]=-1
    usuarios_con_evento = usuarios_con_evento.loc[usuarios_con_evento['event']==evento,['person','date']]\
                            .sort_values(by='date',ascending=True).groupby('person').max().reset_index()
    usuarios_con_evento[nombre]=(fecha_limite-usuarios_con_evento['date']).dt.total_seconds()
    tabla2=usuarios_con_evento.loc[:,['person',nombre]]
    tabla = pd.concat([tabla2,tabla])
    return tabla

In [ ]:
limit_vp = calcularDistancia(event_training,'viewed product','time_to_last_vp')
limit_co = calcularDistancia(event_training,'conversion','time_to_last_co')
limit_ch = calcularDistancia(event_training,'checkout','time_to_last_ch')
limit_le = calcularDistancia(event_training,'lead','time_to_last_le')
limit_ad = calcularDistancia(event_training,'ad compaign hit','time_to_last_ad')
limit_bl = calcularDistancia(event_training,'brand listing','time_to_last_bl')
limit_gl = calcularDistancia(event_training,'geberic listing','time_to_last_gl')
limit_sp = calcularDistancia(event_training,'searched products','time_to_last_sp')
limit_vs = calcularDistancia(event_training,'visited site','time_to_last_vs')
limit_se = calcularDistancia(event_training,'shearch engine hit','time_to_last_se')
limit_st = calcularDistancia(event_training,'staticpage','time_to_last_st')

In [ ]:
last_time_train = limit_vp.merge(limit_co,on='person',how='inner').merge(limit_ch,on='person',how='inner')\
                        .merge(limit_le,on='person',how='inner').merge(limit_ad,on='person',how='inner')\
                            .merge(limit_bl,on='person',how='inner').merge(limit_gl,on='person',how='inner')\
                                .merge(limit_sp,on='person',how='inner').merge(limit_vs,on='person',how='inner')\
                                    .merge(limit_se,on='person',how='inner').merge(limit_st,on='person',how='inner')

In [ ]:
last_time_train.head()

# Ahora para el set de datos test:

In [ ]:
event_test['date'] = pd.to_datetime(event_test['timestamp'])

In [ ]:
limit_vp_test = calcularDistancia(event_test,'viewed product','time_to_last_vp')
limit_co_test = calcularDistancia(event_test,'conversion','time_to_last_co')
limit_ch_test = calcularDistancia(event_test,'checkout','time_to_last_ch')
limit_le_test = calcularDistancia(event_test,'lead','time_to_last_le')
limit_ad_test = calcularDistancia(event_test,'ad compaign hit','time_to_last_ad')
limit_bl_test = calcularDistancia(event_test,'brand listing','time_to_last_bl')
limit_gl_test = calcularDistancia(event_test,'geberic listing','time_to_last_gl')
limit_sp_test = calcularDistancia(event_test,'searched products','time_to_last_sp')
limit_vs_test = calcularDistancia(event_test,'visited site','time_to_last_vs')
limit_se_test = calcularDistancia(event_test,'shearch engine hit','time_to_last_se')
limit_st_test = calcularDistancia(event_test,'staticpage','time_to_last_st')

In [ ]:
last_time_test = limit_vp_test.merge(limit_co_test,on='person',how='inner').merge(limit_ch_test,on='person',how='inner')\
                        .merge(limit_le_test,on='person',how='inner').merge(limit_ad_test,on='person',how='inner')\
                            .merge(limit_bl_test,on='person',how='inner').merge(limit_gl_test,on='person',how='inner')\
                                .merge(limit_sp_test,on='person',how='inner').merge(limit_vs_test,on='person',how='inner')\
                                    .merge(limit_se_test,on='person',how='inner').merge(limit_st_test,on='person',how='inner')

In [ ]:
train_paso_2 = pd.read_csv('train_paso_2.csv')
test_paso_2 = pd.read_csv('test_paso_2.csv')

In [ ]:
train_paso_3 = train_paso_2.merge(last_time_train,on='person',how='inner')
test_paso_3 = test_paso_2.merge(last_time_test,on='person',how='inner')

In [ ]:
train_paso_3.head()

In [ ]:
test_paso_3.head()

In [ ]:
test_paso_3.to_csv('test_paso_3.csv',index=False)
train_paso_3.to_csv('train_paso_3.csv',index=False)